<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Editing-MNIST-example-to-use-COCO-data" data-toc-modified-id="Editing-MNIST-example-to-use-COCO-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Editing MNIST example to use COCO data</a></span><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data</a></span><ul class="toc-item"><li><span><a href="#Original-COCO-ETL" data-toc-modified-id="Original-COCO-ETL-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Original COCO ETL</a></span></li><li><span><a href="#Additions" data-toc-modified-id="Additions-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Additions</a></span></li></ul></li><li><span><a href="#Model-&amp;-Training" data-toc-modified-id="Model-&amp;-Training-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Model &amp; Training</a></span></li><li><span><a href="#Train-model-once-w/-train_n_epochs" data-toc-modified-id="Train-model-once-w/-train_n_epochs-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Train model once w/ <code>train_n_epochs</code></a></span></li><li><span><a href="#Train-model-many-times-w/-train_n_sessions" data-toc-modified-id="Train-model-many-times-w/-train_n_sessions-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Train model many times w/ <code>train_n_sessions</code></a></span></li></ul></li></ul></div>

# Editing MNIST example to use COCO data

## Data

### Original COCO ETL

In [ ]:
from fastai.vision.all import *
from fastcore.test import *
import pandas as pd
import numpy as np
import PIL


### Unzip data ###
path = Path('/home/rory/data/coco2017')
path_items = path.ls()
from zipfile import ZipFile
def unzip(*paths,dest):
    """Unzips files located at posixpaths to dest. Deps: zipfile, pathlib."""
    for path in paths:
        if path.suffix == '.zip':
            print(f"Unzipping: {path} ---> {dest}.")
            with ZipFile(path, 'r') as archive:
                archive.extractall(dest)
    new_items = set(dest.ls()) - set(paths)
    print("Finished. New items in dest:",*new_items,sep='\n')
# unzip(*path_items, dest=path)


### Get ims and annos ###
ims_train, annos_train = get_annotations(path/'annotations/instances_train2017.json')
ims_valid, annos_valid = get_annotations(path/'annotations/instances_val2017.json')
ims_train = list(map(lambda x: Path('train2017/' + x), ims_train))
ims_valid = list(map(lambda x: Path('val2017/' + x), ims_valid))
ims    = ims_train + ims_valid
paths  = list(map(lambda im: Path(path/im), ims))
annos  = annos_train + annos_valid
bboxes = [i[0] for i in annos]
lbls   = [i[1] for i in annos]


### Process annos ###
def transpose(l): return list(zip(*l))
def bbox_area(anno_t): # takes a transposed anno
    b = anno_t[0]
    return((b[2]-b[0])*(b[3]-b[1])) # b-t * l-r
def sort_annos(o): return sorted(transpose(o), key=bbox_area, reverse=True)
sorted_annos = [sort_annos(a) for a in annos]
largest_anno = [i[0] for i in sorted_annos]
largest_bbox = [i[0] for i in largest_anno]
largest_lbl  = [i[1] for i in largest_anno]
is_valid     = [False]*len(ims_train) + [True]*len(ims_valid)


### Store outputs in dfd, df ###
dfd_all = {
    "im":ims,
    "path":paths,
    "annos":annos,
    "bboxes":bboxes,
    "lbls":lbls,
    "sorted_annos":sorted_annos,
    "anno":largest_anno,
    "bbox":largest_bbox,
    "lbl":largest_lbl,
    "is_valid":is_valid}
df_all = pd.DataFrame(dfd_all)

### Additions

Create subset and throw it in a dl.

In [ ]:
### Grab data & grab singletons ###
data = L(zip(paths, largest_lbl))
num_labels = L(len(l) for l in lbls)
singletons = data[num_labels.map(lambda n:n==1)]

### Given a lbl, grab all paths with that label ###
path2lbl = {p:l for p,l in singletons}
def get_lbl(path): return path2lbl[path]
transposed = list(zip(*singletons))
lbl2paths = {l:[p for p in transposed[0] if get_lbl(p) == l]
             for l in set(transposed[1])}

### Grab labels with more than 500 singleton ims ###
lbl_subset = []
for lbl in lbl2paths:
    l = len(lbl2paths[lbl])
    if l > 500: lbl_subset += [lbl]

### Grab ims in label_subset ###
data_subset = L(s for s in singletons if s[1] in lbl_subset)

### Open images as tensors ###
def open_im(imp, size=128):
    im = PIL.Image.open(imp).convert('RGB')
    im = im.resize((size, size))
    t = torch.Tensor(np.array(im))
    t = t.permute(2,0,1).float()/255.0
    return t.reshape([3*size*size])

### Create 1HE encodings ###
n_cls = len(lbl_subset)
encs = []
for i in range(n_cls):
    l = L([0]*n_cls)
    l[i] = 1
    encs += [l]
lbl2enc = {l:tensor(e) for l,e in zip(lbl_subset,encs)}

### Create dataloader ###
ds = [(open_im(path), lbl2enc[lbl]) for path, lbl in data_subset]
dl = DataLoader(ds, batch_size=64, shuffle=True)

(#6856) [(Path('/home/rory/data/coco2017/train2017/000000060760.jpg'), 'toilet'),(Path('/home/rory/data/coco2017/train2017/000000360772.jpg'), 'toilet'),(Path('/home/rory/data/coco2017/train2017/000000191381.jpg'), 'toilet'),(Path('/home/rory/data/coco2017/train2017/000000256668.jpg'), 'toilet'),(Path('/home/rory/data/coco2017/train2017/000000159537.jpg'), 'clock'),(Path('/home/rory/data/coco2017/train2017/000000075051.jpg'), 'toilet'),(Path('/home/rory/data/coco2017/train2017/000000368978.jpg'), 'airplane'),(Path('/home/rory/data/coco2017/train2017/000000078371.jpg'), 'airplane'),(Path('/home/rory/data/coco2017/train2017/000000561100.jpg'), 'airplane'),(Path('/home/rory/data/coco2017/train2017/000000009426.jpg'), 'airplane')...]

## Model & Training

In [ ]:
### Model ###
def init_mod(im_size, n_cls, hidden_params):
    mod = nn.Sequential(
        nn.Linear(im_size,hidden_params),
        nn.ReLU(),
        nn.Linear(hidden_params,n_cls)
    )
    return mod

### Loss ###
def softmax(t):
    if len(t.shape) == 1: return torch.exp(t) / torch.exp(t).sum()
    else:                 return torch.exp(t) / torch.exp(t).sum(dim=1, keepdim=True)
def loss(yp, y):
    return (1 - (y*softmax(yp)).sum(dim=1, keepdim=True)).mean()  # softmax cross entropy loss

In [ ]:
n_cls

10

In [ ]:
### Calculate gradients for use in train_once ###
def calc_grad(x,y,mod):
    yp = mod(x)     # get predictions
    ls = loss(yp,y) # calculate loss
    ls.backward()   # take gradient w.r.t. loss

### Create SGD Stepper; args = (mod.parameters(), lr) ###
class ParamStepper:
    def __init__(self, p, lr): self.p,self.lr = list(p),lr # initialize w/ mod.params & lr
        
    def step(self, *args, **kwargs):                       # take step
        for o in self.p: o.data -= o.grad.data * self.lr
            
    def zero_grad(self, *args, **kwargs):                  # reset grad
        for o in self.p: o.grad = None

### Train parameters by performing SGD on each mini-batch in the dl ###
def train_one_epoch(dl, mod, stepper):
    for xb,yb in dl:           # for every minibatch (xb,yb) in the dataloader:
        calc_grad(xb, yb, mod) # calc grad(loss(mod(xb),yb))
        stepper.step()         # take step
        stepper.zero_grad()    # reset grad

### Get accuracy of mod on a mini-batch ###
def mb_acc(yp,y):
    yp_max,yp_i = torch.max(yp, dim=1, keepdim=True)
    y_max, y_i  = torch.max(y,  dim=1, keepdim=True)
    return (yp_i==y_i).float().mean()
        
### Get accuracy of mod on a dataloader (takes avg of all mbs in dl) ###
def epoch_acc(dl, mod):
    a = [mb_acc(mod(xb), yb) for xb,yb in dl]
    return round(torch.stack(a).mean().item(), 5)          # avg acc over all mini-batches

### Run `train_once` `epochs` times given data `dl`, model `mod`, and stepper `stepper`
def train_n_epochs(dl, mod, stepper, nepochs):
    accs = L()
    for i in range(nepochs):
        print('.',end='')
        train_one_epoch(dl, mod, stepper)
        accs += epoch_acc(dl, mod)
    print('',end='\t')
    return accs

### Perform n training sessions ###
def train_n_sessions(dl, im_size, n_cls, hidden_params, nepochs, lr, nsessions):
    # accuracies and trained models are returned by train_n_sessions for access after training.
    accs = L()
    mods = L()
    # train model and record accs, mods
    print('Progress:',end='\n')
    for i in range(nsessions):
        print(i,end='')
        mod = init_mod(im_size, n_cls, hidden_params)
        stepper = ParamStepper(mod.parameters(), lr)
        accs += train_n_epochs(dl, mod, stepper, nepochs)
        mods += mod
    # restructure mods into nn.Sequential form for end-user convenience
    nmods = len(mods)
    trained_models = L()
    for n in range(nsessions):
        layers = L()
        for j in range(nmods//nsessions): layers.append(mods.pop())
        layers.reverse()
        seqmod = nn.Sequential(*layers.items)
        trained_models += [seqmod]
    print('Done')
    return accs, trained_models

## Train model once w/ `train_n_epochs`

In [ ]:
# params
im_size       = 128*128*3
batch_size    = 64
hidden_params = 30
lr            = .01
nepochs       = 20
n_cls         = len(lbl_subset)

# inits
mod           = init_mod(im_size, n_cls, hidden_params)
stepper       = ParamStepper(mod.parameters(), lr)

# train
accs = train_n_epochs(dl, mod, stepper, nepochs)
print(accs)
print(max(accs))

....................	(#20) [0.28979,0.31076,0.32364,0.33116,0.32827,0.33261,0.33898,0.32812,0.33955,0.33507...]
0.37616


## Train model many times w/ `train_n_sessions`

In [ ]:
# params
path          = untar_data(URLs.MNIST)
n_cls         = 10
im_size       = 28*28
batch_size    = 64*2*2*2
hidden_params = 30
nepochs       = 30
lr            = .1
nsessions     = 4

# inits
# dl = init_data(path, im_size, n_cls, batch_size)

# train
accs,mods = train_n_sessions(dl, im_size, n_cls, hidden_params, nepochs, lr, nsessions)
accs_t = tensor(accs).reshape(nsessions,nepochs)

# print max acc
print("Max accuracy:",round(accs_t.max().item(),4))

# print training sessions
pd.DataFrame(accs_t.numpy())

Progress:
0..............................	1..............................	2..............................	3..............................	Done
Max accuracy: 0.918


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.20321,0.36224,0.64941,0.72600,0.78859,0.79937,0.80784,0.87557,0.88299,0.88825,...,0.91078,0.91226,0.91305,0.91460,0.91491,0.91579,0.91647,0.91694,0.91773,0.91803
1,0.28229,0.41783,0.55677,0.56697,0.72080,0.73564,0.74246,0.74530,0.74886,0.75083,...,0.76214,0.76183,0.76275,0.76358,0.76409,0.76411,0.76448,0.76473,0.76548,0.76562
2,0.29635,0.52273,0.59737,0.77249,0.80325,0.81428,0.82119,0.82421,0.82798,0.82920,...,0.84383,0.84389,0.84510,0.84530,0.84617,0.84669,0.84775,0.84849,0.84820,0.87624
3,0.39058,0.42651,0.64370,0.65673,0.66317,0.66713,0.73243,0.74230,0.74811,0.75052,...,0.76288,0.76335,0.76415,0.76439,0.76494,0.76504,0.76519,0.76547,0.76642,0.77723
